In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio
import tifffile as tiff 
import pickle
from pathlib import Path
from skimage import io
from tensorflow.keras import backend as K

from datetime import datetime
# from deepcell.applications import NuclearSegmentation
# from deepcell.applications import CellTracking


### Insert folder to p - all views inside will be analyzed and results will be saved to different folders

In [ ]:
tif_path = Path('../../Ilan_Data/tif')

In [ ]:
def get_cord_for_crop(tif_files):
    tff = tiff.imread(tif_files[0])
    Y_size = tff.shape[0]
    X_size = tff.shape[1]
    cord_list = []
    cord_list.append([0,int(X_size/2),0,int(Y_size/2)])
    cord_list.append([0,int(X_size/2),int(Y_size/2), Y_size])
    cord_list.append([int(X_size/2),X_size,0,int(Y_size/2)])
    cord_list.append([int(X_size/2),X_size,int(Y_size/2), Y_size])
    return cord_list

def merge_tff_even_crop(tif_files ,cord_list, seq_length = 0):
    if cord_list:
        x1,x2,y1,y2 = cord_list
    if tif_files:
        tff = tiff.imread(tif_files[0])
        tff = tff[y1:y2,x1:x2]
        if len(tff.shape) == 2:
            tff = np.expand_dims(tff, axis=-1)
        tff = np.expand_dims(tff, axis=0)
    if tif_files[1:]:
        for i, other in enumerate(tif_files[1:]):
            if i%2 == 0:
                continue
            othertff = tiff.imread(other)
            othertff = othertff[y1:y2,x1:x2]
            if len(othertff.shape) == 2:
                othertff = np.expand_dims(othertff, axis=-1)
            othertff = np.expand_dims(othertff, axis=0)
            tff = np.concatenate((tff,othertff))

    if seq_length > 0 and seq_length < len(tff):
        tff = tff[0:seq_length,...]
    return tff

In [ ]:
def merge_tff(tif_files , seq_length = 0):
    if tif_files:
        tff = tiff.imread(tif_files[0])
        if len(tff.shape) == 2:
            tff = np.expand_dims(tff, axis=-1)
        tff = np.expand_dims(tff, axis=0)
    if tif_files[1:]:
        for i, other in enumerate(tif_files[1:]):
            othertff = tiff.imread(other)
            if len(othertff.shape) == 2:
                othertff = np.expand_dims(othertff, axis=-1)
            othertff = np.expand_dims(othertff, axis=0)
            tff = np.concatenate((tff,othertff))

    if seq_length > 0 and seq_length < len(tff):
        tff = tff[0:seq_length,...]
    return tff

In [ ]:
def pick_channel(x,channel = 0):
    return x[...,channel:(channel+1)]


In [ ]:
def segmentation(x, mpp =1.24):
    app = NuclearSegmentation()
    y_seg = app.predict(x, image_mpp = mpp)
    return y_seg

In [ ]:
def tracking(x, y_seg):
    tracker = CellTracking()
    tracked_data = tracker.track(np.copy(x), y_seg)
    return tracked_data

In [ ]:
def plot_tff(im1,im2,vmin,vmax):
    fig, ax = plt.subplots(1, 2, figsize=(30, 15))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(im2, cmap='jet', vmin=vmin, vmax=vmax)
    ax[1].set_title('Tracked')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image



In [ ]:
def save_results(x,y_seg, tracked_data, tff_path,well,view):
    centroids = pd.DataFrame(columns = range(x.shape[0]))
    morphologies = pd.DataFrame(columns = range(x.shape[0]))
    embeddings = pd.DataFrame(columns = range(x.shape[0]))

    for cell_id, cell_dict in tracked_data['tracks'].items():
        for i,frame in enumerate(cell_dict['frames']):
            centroids.at[cell_id,frame] = cell_dict['centroid'][i]
            morphologies.at[cell_id,frame] = cell_dict['morphologies'][i]
            embeddings.at[cell_id,frame] = cell_dict['embedding'][i]

    date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    directory = f"Results_{well}_{view}_{date}"
    res_path = tff_path.joinpath(directory)
    os.mkdir(res_path)

    centroids.to_csv(res_path.joinpath('centroids.csv'))
    morphologies.to_csv(res_path.joinpath('morphologies.csv'))
    embeddings.to_csv(res_path.joinpath('embeddings.csv'))

    with open(res_path.joinpath('track_results.pkl'), 'wb') as f:
        pickle.dump(tracked_data, f)

    X = tracked_data['X']  # raw X data
    y = tracked_data['y_tracked']  # tracked y data

    imageio.mimsave(res_path.joinpath('tracks.tif'), [plot_tff(x[i,...,0], y[i,...,0], y.min(), y.max())
                                for i in range(y_seg.shape[0])])
    imageio.mimsave(res_path.joinpath('tracks.gif'), [plot_tff(x[i,...,0], y[i,...,0], y.min(), y.max())
                        for i in range(y_seg.shape[0])])
                                

In [ ]:
tif_files = list(tif_path.glob('*.tif'))
well_dict = {}
for tif_file in tif_files:
    file_name = tif_file.stem.split('_')
    well_name = file_name[2]
    view_name = file_name[3]
    
    if well_name not in well_dict:
        well_dict[well_name] = {}
    if view_name not in well_dict[well_name]:
        well_dict[well_name][view_name] = []

    well_dict[well_name][view_name].append(tif_file)

for well, views in well_dict.items():
    for view , view_tif in views.items():
        print(f'well:{well} , view:{view}, {len(view_tif)}')

In [ ]:
import numpy as np
import cv2
import os

In [ ]:
for well, views in well_dict.items():
    for view , view_tif in views.items():
        print(f'well:{well} , view:{view}, {len(view_tif)}')
        view_tif.sort()
        merged_tff = merge_tff(view_tif)
        merged_tff = merged_tff[5:,...]
        print(type(merged_tff))
        print(merged_tff.shape)
        break
    break
 
width = merged_tff.shape[2]
hieght = merged_tff.shape[1]

well:G4 , view:2, 95
<class 'numpy.ndarray'>
(90, 1040, 1408, 1)


In [ ]:
fps = 6
 
# Syntax: VideoWriter_fourcc(c1, c2, c3, c4) # Concatenates 4 chars to a fourcc code
#  cv2.VideoWriter_fourcc('M','J','P','G') or cv2.VideoWriter_fourcc(*'MJPG)
 
fourcc = cv2.VideoWriter_fourcc(*'DVIX') # FourCC is a 4-byte code used to specify the video codec.
# A video codec is software or hardware that compresses and decompresses digital video. 
# In the context of video compression, codec is a portmanteau of encoder and decoder, 
# while a device that only compresses is typically called an encoder, and one that only 
# decompresses is a decoder. Source - Wikipedia
 
#Syntax: cv2.VideoWriter( filename, fourcc, fps, frameSize )
video = cv2.VideoWriter('test.avi', fourcc, float(fps), (width, hieght))

merged_tff = merged_tff.astype(np.uint8)
for img in merged_tff:
    video.write(img)
 
video.release()

OpenCV: FFMPEG: tag 0x58495644/'DVIX' is not found (format 'avi / AVI (Audio Video Interleaved)')'
[ERROR:0@3114.897] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/videoio/src/cap.cpp (597) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): test.avi in function 'icvExtractPattern'




In [ ]:
for well, views in well_dict.items():
    for view , view_tif in views.items():
        print(f'well:{well} , view:{view}, {len(view_tif)}')
        view_tif.sort()
        cord_list = get_cord_for_crop(view_tif)

In [ ]:
for well, views in well_dict.items():
    for view , view_tif in views.items():
        print(f'well:{well} , view:{view}, {len(view_tif)}')
        view_tif.sort()
        cord_list = get_cord_for_crop(view_tif)
        for i,cord in enumerate(cord_list):
            print(cord)
            merged_tff = merge_tff_even_crop(view_tif,cord) # , seq_length = 30
            merged_tff = merged_tff[5:,...]
            print(merged_tff.shape)
            # tff = pick_channel(tff,0)
            print("Segmentation")
            seg_tff = segmentation(merged_tff, mpp=1.24) 
            #(Drop if is empty)
            print("Tracking")
            track_tff = tracking(merged_tff,seg_tff)
            save_results(merged_tff,seg_tff,track_tff,tif_path,well,f'{view}_crop_{i}')

        

In [ ]:
# for well, views in well_dict.items():
#     for view , view_tif in views.items():
#         print(f'well:{well} , view:{view}, {len(view_tif)}')
#         merged_tff = merge_tff(view_tif) #, seq_length= 30
#         merged_tff = merged_tff[5:,...]
#         # tff = pick_channel(tff,0)
#         print("Segmentation")
#         seg_tff = segmentation(merged_tff, mpp=1.24) 
#         print("Tracking")
#         track_tff = tracking(merged_tff,seg_tff)
#         save_results(merged_tff,seg_tff,track_tff,tif_path,well,view)
#         del track_tff
#         del seg_tff

In [ ]:
merged_tff.shape

(48, 1040, 1408, 1)

In [ ]:
seg_tff.shape

(48, 1040, 1408, 1)

In [ ]:
merged_tff = merged_tff[18:,...]
seg_tff = seg_tff[18:,...]

In [ ]:
print(merged_tff.shape , seg_tff.shape)

(30, 1040, 1408, 1)

(30, 1040, 1408, 1)

In [ ]:
track_tff = tracking(merged_tff,seg_tff)
save_results(merged_tff,seg_tff,track_tff,tif_path,well,view)

In [ ]:
embeddings = pd.DataFrame(columns = range(merged_tff.shape[0]))

for cell_id, cell_dict in track_tff['tracks'].items():
    for i,frame in enumerate(cell_dict['frames']):
        embeddings.at[cell_id,frame] = cell_dict['embedding'][i]